In [1]:
!pip install -U transformers==4.30.0

import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
import torch
import torch.nn as nn
from torch import nn, Tensor
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
from torch.utils.data import Dataset, DataLoader
import random
from sklearn.metrics import r2_score, mean_squared_error
from torch.optim.lr_scheduler import StepLR
from transformers import InformerForPrediction, InformerConfig

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
path = "drive/MyDrive/feature1.csv"
df = pd.read_csv(path)

In [ ]:
class vessel_data(Dataset):
    def __init__(self, train = True, test=False, train_test_split = 0.7, rand_seed=1):
        ##########################inputs##################################
        #data_dir(string) - directory of the data#########################
        #size(int) - size of the images you want to use###################
        #train(boolean) - train data or test data#########################
        #train_test_split(float) - the portion of the data for training###
        #augment_data(boolean) - use data augmentation or not#############
        super(vessel_data, self).__init__()
        # todo
        #initialize the data class
        trips = list(df.trip_id.unique())
        self.train = train
        self.sequence_length = sequence_length
        self.prediction_horizon = prediction_horizon

        # train_test_split
        random.seed(rand_seed)
        train_size = int(np.ceil(len(trips)*train_test_split))
        train_trips = random.sample(trips, k=train_size)
        if train:
            self.trips_id = train_trips
            # self.trips_id = train_trips[0:2]
        else:
            test_trips = [ x for x in trips if x not in train_trips]
            valid_trips = random.sample(test_trips, k = int(np.ceil(len(trips)*((1-train_test_split)*.7))))
            if test==False:
                self.trips_id = valid_trips
            else:
                self.trips_id = [ x for x in test_trips if x not in valid_trips]
        # self.starting_dict = self.get_starting(df)

    # convert a df to tensor
    def df_to_tensor(self, df):
        numpy = df.to_numpy(dtype="double")
        if torch.cuda.is_available():
            device = torch.device('cuda:0')
        else:
            device = torch.device('cpu')
        # return torch.from_numpy(df.values.astype("float")).float().to(device)
        return torch.from_numpy(numpy).float().to(device)


    def __getitem__(self, idx):
        #load corresponding trip id from index idx of your data
        trip_id = (self.trips_id[idx//2])
        # start = self.starting_dict[trip_id]
        # start_idx = self.starting_dict[trip_id] + (idx%3)*(self.prediction_horizon+self.sequence_length)
        data = df[df.trip_id==trip_id].reset_index(drop=True)
        shifted_data = shifted_df[df.trip_id==trip_id].reset_index(drop=True)
        starting = 0
        if (idx%2):
            starting = len(data)-91

        starting = 0
        data = data.iloc[starting:starting+91]

        values = data[y_cols]
        time_features = data[time_feature + dynamic_real_feature]
        static_categorical_features = data.iloc[0][static_categorical_feature]
        future_time_features = shifted_data.iloc[starting:starting+91][time_feature + dynamic_real_feature]
        actions = data[["SPEED", "HEADING", "MODE", "turn"]]
        return self.df_to_tensor(values), self.df_to_tensor(time_features), self.df_to_tensor(static_categorical_features), self.df_to_tensor(future_time_features), self.df_to_tensor(actions)

    def __len__(self):
        return len(self.trips_id)*2

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
# device = torch.device('cpu')

batch_size=256
sequence_length = 25
context_length = 24
prediction_horizon = 5
# criterion = Weighted_Loss()

time_feature = ["Time2"]
dynamic_real_feature = [ "SPEED", "HEADING", "MODE", "turn", "acceleration",
       'current', 'rain', 'snowfall', 'wind_force', 'wind_direc', "resist_ratio"]
static_categorical_feature = ["is_weekday", 'direction',"season", "hour"]
y_cols = ["FC2", "SOG2"]

config = InformerConfig.from_pretrained("huggingface/informer-tourism-monthly", prediction_length=prediction_horizon,
        context_length=context_length, input_size=len(y_cols), num_time_features=len(time_feature),
        num_dynamic_real_features = len(dynamic_real_feature), num_static_real_features = len(static_categorical_feature),
        lags_sequence=[1], num_static_categorical_features=0, feature_size=22)
model = InformerForPrediction.from_pretrained("huggingface/informer-tourism-monthly",
                                                           config=config, ignore_mismatched_sizes=True).to(device)


Some weights of the model checkpoint at huggingface/informer-tourism-monthly were not used when initializing InformerForPrediction: ['model.embedder.embedders.0.weight']
- This IS expected if you are initializing InformerForPrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing InformerForPrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of InformerForPrediction were not initialized from the model checkpoint at huggingface/informer-tourism-monthly and are newly initialized because the shapes did not match:
- model.encoder.embed_positions.weight: found shape torch.Size([48, 32]) in the checkpoint and torch.Size([29, 32]) in the model instantiated
- model.decoder.embed_posi

In [ ]:
class GRU_update(nn.Module):
    def __init__(self, input_size, hidden_size=1, output_size=4, num_layers=1, prediction_horizon=5):
        super().__init__()
        self.h = prediction_horizon
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.mlp = nn.Sequential( nn.ReLU(),
                                  nn.Linear(hidden_size, 2048),
                                  nn.Dropout(0.2),
                                  nn.ReLU(),
                                  nn.Linear(2048, output_size))
        self.hx_fc = nn.Linear(2*hidden_size, hidden_size)

    def forward(self, predicted_values, past_time_features):
        xy = torch.zeros(size=(past_time_features.shape[0], 1, self.output_size)).float().to(device)
        hx = past_time_features.reshape(-1, 1, self.hidden_size)
        hx = hx.permute(1, 0, 2)
        out_wp = list()
        for i in range(self.h):
            ins = torch.cat([xy, predicted_values[:, i:i+1, :]], dim=1) # x
            hx, _ = self.gru(ins, hx.contiguous())
            hx = hx.reshape(-1, 2*self.hidden_size)
            hx = self.hx_fc(hx)
            d_xy = self.mlp(hx).reshape(-1, 1, self.output_size) #control v4
            hx = hx.reshape(1, -1, self.hidden_size)
            # print("dxy", d_xy)
            xy = xy + d_xy
            # print("xy plused", xy)
            out_wp.append(xy)
        pred_wp = torch.stack(out_wp, dim=1).squeeze(2)
        return pred_wp

In [ ]:
trainset = vessel_data(train = True, train_test_split = 0.8, rand_seed=1)
trainloader = DataLoader(trainset, batch_size = batch_size, shuffle=True, drop_last=True)

validset = vessel_data(train = False, train_test_split = 0.8, rand_seed=1)
validloader = DataLoader(validset, batch_size = batch_size, drop_last=True)

testset = vessel_data(train = False, test=True, train_test_split = 0.8, rand_seed=1)
testloader = DataLoader(testset, batch_size = batch_size, drop_last=True)

In [ ]:
model = model.float()

In [ ]:
gru = GRU_update(2, hidden_size=300, output_size=2, num_layers=1, prediction_horizon=5).to(device)

In [ ]:
#  train the model
def train_model(trainloader, testloader, model, model_name = "transformer"):
    for epoch in range(n_epochs):
        epoch_loss = 0
        mses = []
        r2s = []
        losses = []
        gru.train()
        model.train()
        for _, data in enumerate(trainloader):
            values, time_features, static_real_features, future_features, actions  = data
            for t in range(sequence_length, values.shape[1]-prediction_horizon):
                # for i in range(prediction_horizon):
                # if t!=25:
                #     values[:, t-1, :] =  predicted[:, 0, :]
                #     acceleration = predicted[:,0,1]- values[:, t-2, 1]
                #     time_features[:, t-1, 6] = acceleration
                # future_time_features = time_features[:, t-prediction_horizon:t]
                # future_time_features[:, 0, 1:5] = time_features[:, t, 1:5]  # from speed to turn
                # future_time_features[:, :, 0] = future_time_features[:, :, 0]+prediction_horizon
                # future_time_features[:, 0, 1:5] = actions[:, t, 0:4]
                # # future_values[:, :, 0] = values[:, t+i:t+prediction_horizon+i, 0]
                # future_values = values[:, t:t+prediction_horizon]
                # past_values = values[:, t-sequence_length: t]
                # past_time_features = time_features[:, t-sequence_length: t]

                i = (t-sequence_length)%prediction_horizon
                # print(i)
                if i==0:
                    values_tmp = torch.clone(values[:, t-sequence_length:t+prediction_horizon*2])
                    time_features_tmp = torch.clone(time_features[:, t-sequence_length:t+prediction_horizon*2])
                    # print(values_tmp.shape, time_features_tmp.shape)
                else:
                    predicted = predicted.detach()
                    values_tmp[:, sequence_length-1 + i, :] =  predicted[:, 0, :]
                    acceleration = predicted[:, 0, 1]- values_tmp[:, sequence_length-2 + i, 1]
                    time_features_tmp[:, sequence_length-1 + i, 5] = acceleration
                future_time_features = time_features_tmp[:, sequence_length+i-prediction_horizon: sequence_length+i]
                future_time_features[:, 0, 1:5] = time_features_tmp[:, sequence_length + i, 1:5]  # from speed to turn
                future_time_features[:, :, 0] = future_time_features[:, :, 0]+prediction_horizon/120

                future_values = values_tmp[:, sequence_length+i : sequence_length+i+prediction_horizon]
                past_values = values_tmp[:, i: sequence_length+i]
                past_time_features = time_features_tmp[:, i: sequence_length+i]

                # print(past_values.shape, past_time_features.shape)
                # print(future_values.shape, future_time_features.shape)

                # print(past_values.shape, past_time_features.shape)
                past_observed_mask = torch.ones(past_values.shape).to(device)
                optimizer1.zero_grad()
                optimizer2.zero_grad()
                tf_out = model(past_values=past_values, past_time_features=past_time_features, static_real_features=static_real_features,
                            past_observed_mask=past_observed_mask, future_values=future_values, future_time_features=future_time_features, output_hidden_states=True)
                loss1 = tf_out.loss
                loss1.backward()
                with torch.no_grad():
                    predicted_tf = model.generate(past_values=past_values, past_time_features=past_time_features, static_real_features=static_real_features,
                                  past_observed_mask=past_observed_mask, future_time_features=future_time_features).sequences.mean(dim=1)
                predicted = gru(predicted_values = predicted_tf, past_time_features=past_time_features)

                loss2 = criterion(predicted, future_values)

                # loss = prediction.loss
                epoch_loss += loss2.item()
                loss2.backward()
                optimizer1.step()
                optimizer2.step()
                # predicted = model.generate(past_values=past_values, past_time_features=past_time_features, static_real_features=static_real_features,
                            # past_observed_mask=past_observed_mask, future_time_features=future_time_features).sequences.mean(dim=1)
        # scheduler.step()
        epoch_loss = epoch_loss / len(trainloader) / (91-25)
        print('Epoch %d / %d --- Loss: %.8f' % (epoch, n_epochs, epoch_loss))
        # mse, r2 = evaluate_model(testloader, model)
        mse, r2 = evaluate_model(testloader, model)
        print('valid MSES: {}'.format(mse))
        print('valid R2s:  {}'.format(r2))

        mses.append(mse)
        r2s.append(r2)
        losses.append(epoch_loss)
        path = "drive/MyDrive/{}_checkpoint{}.pt".format(model_name, epoch)
        torch.save(model.state_dict(), path)
        path = "drive/MyDrive/{}_checkpoint{}_gru.pt".format(model_name, epoch)
        torch.save(gru.state_dict(), path)
    # path = "drive/MyDrive/{}_final.pt".format(model_name, n_epochs)
    # torch.save(model.state_dict(), path)
    return mses, r2s, losses

In [ ]:
def evaluate_model(testloader, model, mode="valid", i=0):
    gru.eval()
    model.eval()
    predictions = []
    actuals = []
    mses = [[] for _ in range(2)]
    r2s =[[] for _ in range(2)]
    with torch.no_grad():
      for _, data in enumerate(testloader):
            outputs = None
            act = []
            pred = []
            values, time_features, static_real_features, future_features, actions  = data
            values_tmp = torch.clone(values)
            for t in range(sequence_length, values.shape[1]-prediction_horizon):
                future_values = values[:, t : t+prediction_horizon]
                if outputs != None:
                    predicted = outputs.detach()
                    values_tmp[:, t-1, :] =  predicted[:, 0, :]
                    acceleration = predicted[:, 0, 1]- values_tmp[:, t-2, 1]
                    time_features[:, t-1, 5] = acceleration

                future_time_features = time_features[:, t-prediction_horizon: t]
                future_time_features[:, 0, 1:5] = time_features[:, t, 1:5]  # from speed to turn
                future_time_features[:, :, 0] = future_time_features[:, :, 0]+prediction_horizon/120

                future_values = values[:, t : t+prediction_horizon]
                past_values = values_tmp[:, t-sequence_length: t]
                past_time_features = time_features[:, t-sequence_length: t]

                past_observed_mask = torch.ones(past_values.shape).to(device)
                # return past_values, past_time_features, static_categorical_features, past_observed_mask, future_time_features
                # outputs = model.generate(past_values=past_values, past_time_features=past_time_features, static_real_features=static_real_features,
                                        # past_observed_mask=past_observed_mask, future_time_features=future_time_features).sequences.mean(dim=1)
                predicted_tf = model.generate(past_values=past_values, past_time_features=past_time_features, static_real_features=static_real_features,
                                        past_observed_mask=past_observed_mask, future_time_features=future_time_features).sequences.mean(dim=1)
                outputs = gru(predicted_tf, past_time_features)
                yhat = outputs[:, 0, :].detach().cpu().numpy()
                actual = values[:, t, :].detach().cpu().numpy()

                # store
                act.append(actual)
                pred.append(yhat)
            act = np.stack(act, axis=1)
            pred = np.stack(pred, axis=1)
            for i in range(2):
                mse = mean_squared_error(act[:, i], pred[:, i])
                mses[i].append(mse)
                actual_1 = act[:, i].swapaxes(1, 0).reshape([-1, 1])
                yhat_1 = pred[:, i].swapaxes(1, 0).reshape([-1, 1])
                r2 = r2_score(actual_1, yhat_1)
                r2s[i].append(r2)
            predictions.append(pred)
            actuals.append(act)
    mses = [ sum(x)/len(x) for x in mses]
    r2s = [ sum(x)/len(x) for x in r2s]

    if mode=="test":
      return mses, r2s, [actuals, predictions]
    # calculate mse
    return mses, r2s
    #actuals[:20], predictions[:20]

In [ ]:
model.load_state_dict(torch.load("drive/MyDrive/gru_3_checkpoint8.pt"))
gru.load_state_dict(torch.load("drive/MyDrive/gru_3_checkpoint8_gru.pt"))


<All keys matched successfully>

In [ ]:
iter = 0

In [ ]:
iter = iter+1
criterion = torch.nn.MSELoss()
print(iter)
n_epochs = 30
optimizer1 = torch.optim.Adam(model.parameters(), lr= 1e-5)
optimizer2 = torch.optim.Adam(gru.parameters(), lr= 1e-5)

# scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
mses, r2s, losses = train_model(trainloader, validloader, model, "gru_{}".format(iter))

In [ ]:
mses, r2s, [actuals, predictions ] = evaluate_model(testloader, model, mode="test")